In [1]:
%pip install numpy
%pip install pandas
%pip install sklearn


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2367 sha256=a096c2db12f9dcf874307ff3f521bfe426f3c7e2604ef222216d001a8a3f685d
  Stored in directory: c:\users\aditya kumar\appdata\local\pip\cache\wheels\f8\e0\3d\9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Program Files\Python39\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ADITYA KUMAR\\AppData\\Local\\Temp\\pip-install-801upfdc\\sklearn_fa6182ff0e904974b04b1e5d916b9033\\setup.py'"'"'; __file__='"'"'C:\\Users\\ADITYA KUMAR\\AppData\\Local\\Temp\\pip-install-801upfdc\\sklearn_fa6182ff0e904974b04b1e5d916b9033\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\ADITYA KUMAR\AppData\Local\Temp\pip-pip-egg-info-jsekd934'
         cwd: C:\Users\ADITYA KUMAR\AppData\Local\Temp\pip-install-801upfdc\sklearn_fa6182ff0e904974b04b1e5d916b9033\
    Complete output (7 lines):
    Traceback (most recent call last):
      File "<string>", line 1,

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("drug-adverse-effect.csv")
df.head()

,safetyreportid,primarysourcecountry,serious,seriousnessdeath,seriousnesslifethreatening,seriousnesshospitalization,seriousnessdisabling,seriousnesscongenitalanomali,seriousnessother,patientonsetage,patientonsetageunit,patientsex,reactionmeddrapt,medicinalproduct,activesubstancename
0,20401550,US,2,2.0,2.0,2.0,2.0,2.0,2.0,43.0,801.0,2.0,"Cough, Drug specific antibody present, Drug in...",HUMIRA,ADALIMUMAB
1,20719436,US,2,2.0,2.0,2.0,2.0,2.0,2.0,73.0,801.0,2.0,Ear infection,COSENTYX,SECUKINUMAB
2,20983728,US,1,2.0,2.0,2.0,2.0,2.0,1.0,26.0,801.0,2.0,"Autism spectrum disorder, Mucous stools, Gastr...",DUPIXENT,DUPILUMAB
3,20401631,US,2,2.0,2.0,2.0,2.0,2.0,2.0,49.0,801.0,2.0,"Device issue, Injury associated with device",HUMIRA,ADALIMUMAB
4,20719827,US,1,2.0,2.0,2.0,2.0,2.0,1.0,69.0,801.0,1.0,Colorectal cancer,ZANTAC,RANITIDINE HYDROCHLORIDE


In [4]:

df.describe()

,serious,seriousnessdeath,seriousnesslifethreatening,seriousnesshospitalization,seriousnessdisabling,seriousnesscongenitalanomali,seriousnessother,patientonsetage,patientonsetageunit,patientsex
count,12000.000000,11476.000000,11456.000000,11580.000000,11416.000000,11420.000000,11666.000000,6894.000000,6893.000000,10368.00000
mean,1.430583,1.918961,1.982804,1.840155,1.988174,1.998511,1.562661,118.922831,801.006383,1.59973
std,0.495179,0.272906,0.130008,0.366478,0.108105,0.038556,0.496079,1262.466732,0.230357,0.49037
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,800.000000,0.00000
25%,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,44.000000,801.000000,1.00000
50%,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,58.000000,801.000000,2.00000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,69.000000,801.000000,2.00000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,33649.000000,804.000000,2.00000


In [5]:
df.shape

(12000, 15)

In [7]:
df.isna().sum()

safetyreportid                     0
primarysourcecountry             107
serious                            0
seriousnessdeath                 524
seriousnesslifethreatening       544
seriousnesshospitalization       420
seriousnessdisabling             584
seriousnesscongenitalanomali     580
seriousnessother                 334
patientonsetage                 5106
patientonsetageunit             5107
patientsex                      1632
reactionmeddrapt                   0
medicinalproduct                   0
activesubstancename               41
dtype: int64

In [8]:
new_df = df[["safetyreportid", "patientonsetage", "patientsex", "medicinalproduct", "activesubstancename", "reactionmeddrapt"]].copy()

In [9]:
new_df.head()

,safetyreportid,patientonsetage,patientsex,medicinalproduct,activesubstancename,reactionmeddrapt
0,20401550,43.0,2.0,HUMIRA,ADALIMUMAB,"Cough, Drug specific antibody present, Drug in..."
1,20719436,73.0,2.0,COSENTYX,SECUKINUMAB,Ear infection
2,20983728,26.0,2.0,DUPIXENT,DUPILUMAB,"Autism spectrum disorder, Mucous stools, Gastr..."
3,20401631,49.0,2.0,HUMIRA,ADALIMUMAB,"Device issue, Injury associated with device"
4,20719827,69.0,1.0,ZANTAC,RANITIDINE HYDROCHLORIDE,Colorectal cancer


In [10]:
new_df.isna().sum()

safetyreportid            0
patientonsetage        5106
patientsex             1632
medicinalproduct          0
activesubstancename      41
reactionmeddrapt          0
dtype: int64